## [Allstate Claims Severity](https://goo.gl/1DwHVy) -- Predictions using machine learning:

### Author: Dr. Rahul Remanan, CEO and  Chief Imagination Officer [Moad Computer](https://www.moad.computer)

The [Allstate Corporation](https://en.wikipedia.org/wiki/Allstate) is the one of the largest insurance providers in the United States and one of the largest that is publicly held. The company also has personal lines insurance operations in Canada. Allstate was founded in 1931 as part of Sears, Roebuck and Co., and was spun off in 1993.[1](https://goo.gl/ce2JJ2) The company has had its headquarters in Northfield Township, Illinois, near Northbrook since 1967.[2](https://goo.gl/oX4kfZ),[3](https://goo.gl/mcTd3y)

As part of Allstate's ongoing efforts to develop automated methods of predicting the cost, and hence severity, of claims, they releasd a claims severity assessment dataset on Kaggle.[4](https://goo.gl/1DwHVy) In this challenge, datascientists were invited to show off their creativity and flex their technical chops by creating an algorithm which accurately predicts claims severity. The goal of this challenge was to help aspiring competitors demonstrate their insight into better ways of predicting claims severity.

We will be using this dataset to build a deep neural network model using  tensorflow and keras.

Users can run this notebook in [Jomiraki Cloud AI instance, with a subscription](https://www.moad.computer/store/p24/Jomiraki_datasci) or in try it for free using [Google CoLab](https://goo.gl/mGu4bH).

#### To launch this example notebook instance in Jomiraki:

* 1) $ git clone https://github.com/rahulremanan/HIMA
* 2) From Jupyter home page, navigate to /HIMA/examples/Notebooks/06_Kaggle_Allstate_Claims_Severity/notebooks/
* 3) Launch the notebook with a python 3 kernel.


## Part 04 -- Deep neural network model using [Keras](https://keras.io/):

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import sklearn
import keras
from __future__ import absolute_import, division, print_function
import keras
from keras.optimizers import SGD,                           \
                             RMSprop,                       \
                             Adagrad
from keras.callbacks import EarlyStopping,   \
                            ModelCheckpoint, \
                            ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
print('The scikit-learn version is {}.'.format(sklearn.__version__))

### Load the training and test files:

In [ ]:
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
print('training: ', df_train.shape)
print('test: ', df_test.shape)

### Data pre-processing:

Convert the structured data into numpy arrays and separate features/targets.

In [ ]:
training_samples = df_train.as_matrix()
training_targets = training_samples[:,-1]
training_samples = training_samples[:,1:-1]

In [ ]:
test_samples = df_test.as_matrix()
test_samples = test_samples[:,1:]

#### Distribution histogram of target values from training data:

In [ ]:
gen_hist_plot = False
if gen_hist_plot:
    plt.hist(training_targets[np.where(training_targets < 15000)], 
             bins = 200, 
             normed=True)
    plt.grid(True)
    plt.xlabel('Target Value')
    plt.ylabel('Normalized Frequency')
    plt.title('Distribution of target values from training set.')
    plt.show()

#### Encode the labels of the categorical data:

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
# [0:116]
allLabels = np.concatenate( ( training_samples[:, 0:116].flat , test_samples[:, 0:116].flat ) )
le.fit( allLabels )
del allLabels

In [ ]:
print_labels = False
if print_labels:
    print(le.classes_)

#### Transform the labels to int values:

In [ ]:
for colIndex in range(116):
    training_samples[:, colIndex] = le.transform(training_samples[:, colIndex])
    test_samples[:, colIndex] = le.transform( test_samples[:, colIndex] )

### Build a deep neural network:

Use keras and tensorflow to build a deep neural network predictor.

In [ ]:
dropout = 0.7
activation = 'elu' # 'relu' or 'elu'
unit_size = 20
input_dim = 130

In [ ]:
def dnn_model(unit_size=None, 
              dropout=None, 
              activation=None, 
              input_dim=None):
    model = Sequential()
    model.add(Dense(unit_size*50, 
                    input_dim=input_dim, 
                    kernel_initializer='normal', 
                    activation=activation))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout))
    
    model.add(Dense(unit_size*50, 
                    kernel_initializer='normal', 
                    activation=activation))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout))
    
    model.add(Dense(unit_size*50, 
                    kernel_initializer='normal', 
                    activation=activation))
    model.add(Dropout(dropout))
    
    model.add(Dense(unit_size*50, 
                    kernel_initializer='normal', 
                    activation=activation))
    model.add(Dropout(dropout))
    
    model.add(Dense(unit_size*25, 
                    kernel_initializer='normal', 
                    activation=activation))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout))
    
    model.add(Dense(unit_size*25, 
                    kernel_initializer='normal', 
                    activation=activation))
    model.add(Dropout(dropout))
    
    model.add(Dense(unit_size*25, 
                    kernel_initializer='normal', 
                    activation=activation))
    model.add(Dropout(dropout))
    
    model.add(Dense(unit_size*5, 
                    kernel_initializer='normal', 
                    activation=activation))
    model.add(Dropout(dropout))
    
    model.add(Dense(unit_size, 
                    kernel_initializer='normal', 
                    activation=activation))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout))
    
    model.add(Dense(unit_size, 
                    kernel_initializer='normal', 
                    activation=activation))
    model.add(Dropout(dropout))
    
    model.add(Dense(1, kernel_initializer='normal'))

    return model

### Build compile and summarize the model:

In [ ]:
model = dnn_model(unit_size=unit_size, dropout=dropout, activation=activation, input_dim=input_dim)

In [ ]:
model.summary()

In [ ]:
DEFAULT_OPTIMIZER = 'adam'

optimizer_val = ada
lr = 1e-5

if optimizer_val.lower() == 'sgd' :
    optimizer = SGD(lr=lr, decay=decay, momentum=1, nesterov=True)
    print ("Using SGD as the optimizer ...")
elif optimizer_val.lower() == 'rms' or optimizer_val.lower() == 'rmsprop':
    optimizer = RMSprop(lr=lr, rho=0.9, epsilon=1e-08, decay=decay)
    print ("Using RMSProp as the optimizer ...")
elif optimizer_val.lower() == 'ada':
    optimizer = Adagrad(lr=lr, epsilon=1e-08, decay=decay)
    print ("Using Adagrad as the optimizer ...")
else:
      optimizer = DEFAULT_OPTIMIZER

In [ ]:
model.compile(optimizer=optimizer, 
              loss='mean_squared_error', 
              metrics=['accuracy'])

In [ ]:
filepath = './model/Allstate_claims_severity_DNN.h5'

checkpointer = ModelCheckpoint(filepath, 
                               verbose=1,  
                               save_best_only=False)
learning_rate_reduction = ReduceLROnPlateau(monitor='loss', 
                                            patience=2,
                                            mode = 'min',
                                            min_delta=1e-4, 
                                            cooldown=1,
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=lr*1e-2)

### Creating regression estimator:

In [ ]:
scale = StandardScaler()
X_train = scale.fit_transform(training_samples)
Y_train = training_targets

### Train the model:

In [ ]:
load_saved_model = True

if load_saved_model and os.path.exists(filepath):
    model = keras.models.load_model(filepath)
    print ("Loaded saved model ...")

In [ ]:
log = model.fit(X_train, Y_train, 
                epochs=20, 
                batch_size=50, 
                verbose=1,
                callbacks=[learning_rate_reduction, 
                           checkpointer])

In [ ]:
model.save(filepath)

### Generating predictions formatted for Kaggle submission:

In [ ]:
X_test = scale.transform(test_samples)
pred_targets = model.predict(X_test)

### Save results to csv file:

In [ ]:
df_res = pd.DataFrame(df_test, columns=['id'])
df_res['loss'] = pred_targets
print(df_res.iloc[0])
df_res.to_csv('DNN_result.csv', index=False)